In [1]:
# Step 1 - Read all the log files
folder_path = r"C:\Users\Shobhit\OneDrive\Desktop\EDA\Scripting\scripting_project\logs"

import os
for file in sorted(os.listdir(folder_path)):
    if file.endswith(".log"):
        print(f"Reading file: {file}")
        with open(os.path.join(folder_path, file), 'r') as log_file:
            print(f"Contents of {file}:")
            print(log_file.read()) # 

Reading file: log1_sim_vcs.log
Contents of log1_sim_vcs.log:
# VCS Simulation Log - Design: ALU, Date: 2025-08-12
Compiling verilog modules...
Compiling module alu.v
Warning-[UNUSED] Unused signal detected in module alu.v: signal temp_sum
Compiling module tb_alu.v
Info: Compilation successful. Starting simulation...

Simulation started at time 0ns
[0ns] Reset deasserted
[5ns] Applying stimulus: A=5, B=3, OPCODE=ADD
[15ns] Output Y=8, PASS
[25ns] Applying stimulus: A=2, B=4, OPCODE=MUL
[35ns] Output Y=8, PASS
[45ns] Applying stimulus: A=7, B=2, OPCODE=DIV
Error-[DIV0] Division by zero detected in alu.v at time 45ns
[55ns] Applying stimulus: A=9, B=1, OPCODE=SUB
[65ns] Output Y=8, FAIL - Expected 8, Got 9
[75ns] Simulation complete

Warning-[COVLOW] Functional coverage is below threshold (65%)
Simulation completed with 2 Errors, 2 Warnings

Reading file: log2_sim_modelsim.log
Contents of log2_sim_modelsim.log:
# ModelSim Simulation Log - Design: FIFO, Date: 2025-08-12
** Compiling fifo.v

In [7]:
#2 Parsing the log files for specific information
import os
import re
import pandas as pd

folder_path = r"C:\Users\Shobhit\OneDrive\Desktop\EDA\Scripting\scripting_project\logs"

# Storage for parsed data
parsed_data = []

for file in sorted(os.listdir(folder_path)):
    if file.endswith(".log"):
        with open(os.path.join(folder_path, file), 'r') as f:
            content = f.read()

        
        design_match = re.search(r"Design:\s*(.*?),", content)
        design = design_match.group(1) if design_match else "Unknown"

        # Simulation Logs
        if "Simulation" in content:  
            simulator = "VCS" if "VCS Simulation" in content else "ModelSim"
            
            # Extracting errors, warnings
            errors_match = re.search(r"Errors?:\s*(\d+)", content)
            warnings_match = re.search(r"Warnings?:\s*(\d+)", content)
            errors = int(errors_match.group(1)) if errors_match else 0
            warnings = int(warnings_match.group(1)) if warnings_match else 0

            # Extracting coverage %
            coverage_match = re.search(r"coverage.*?(\d+)%", content, re.IGNORECASE)
            coverage = int(coverage_match.group(1)) if coverage_match else None

            # Extracting all error messages
            error_messages = re.findall(r"Error.*", content)
            error_messages = "; ".join(error_messages)

            parsed_data.append({
                "File": file,
                "Type": "Simulation",
                "Tool": simulator,
                "Design": design,
                "No. of Errors": errors,
                "Warnings": warnings,
                "Coverage (%)": coverage,
                "Error Messages": error_messages
            })

        # STA logs
        elif "PrimeTime" in content:  
            # Extracting clock period
            clock_match = re.search(r"Clock period set to\s*([\d.]+)", content)
            clock_period = float(clock_match.group(1)) if clock_match else None	
            # Extracting count of setup and hold violations
            setup_violations = len(re.findall(r"Slack \(setup\) = .*VIOLATED", content))
            hold_violations = len(re.findall(r"Slack \(hold\) = .*VIOLATED", content))
            # Extracting worst setup slack values
            worst_setup_match = re.findall(r"Slack \(setup\) = ([\-\d.]+)", content)
            worst_setup_slack = min(map(float, worst_setup_match)) if worst_setup_match else None
            # Extracting worst hold slack values
            worst_hold_match = re.findall(r"Slack \(hold\) = ([\-\d.]+)", content)
            worst_hold_slack = min(map(float, worst_hold_match)) if worst_hold_match else None

            parsed_data.append({
                "File": file,
                "Type": "STA",
                "Tool": "PrimeTime",
                "Design": design,
                "No. of Setup Violations": setup_violations,
                "No. of Hold Violations": hold_violations,
                "Worst Setup Slack": worst_setup_slack,
                "Worst Hold Slack": worst_hold_slack,
            })

# Create DataFrame
df = pd.DataFrame(parsed_data)

# Setting Nan as 0 for computing rank score
df[["No. of Errors", "No. of Setup Violations", "No. of Hold Violations"]] = ( df[["No. of Errors", "No. of Setup Violations", "No. of Hold Violations"]].fillna(0))
# Calculating Rank Score
df["Rank Score"] = (df["No. of Errors"] + df["No. of Setup Violations"] + df["No. of Hold Violations"])
# Sorting on the basis of Rank Score
df = df.sort_values(by="Rank Score", ascending=False)

# Save to CSV
csv_path = os.path.join(folder_path, "parsed_results.csv")
df.to_csv(csv_path, index=False)

df


,File,Type,Tool,Design,No. of Errors,Warnings,Coverage (%),Error Messages,No. of Setup Violations,No. of Hold Violations,Worst Setup Slack,Worst Hold Slack,Rank Score
3,log4_sta_primetime.log,STA,PrimeTime,ALU,0.0,NaN,NaN,NaN,1.0,1.0,-0.23,-0.05,2.0
4,log5_sta_primetime.log,STA,PrimeTime,FIFO,0.0,NaN,NaN,NaN,2.0,0.0,-0.50,0.10,2.0
1,log2_sim_modelsim.log,Simulation,ModelSim,FIFO,1.0,2.0,70.0,"Error: FIFO underflow at time 80ns; Errors: 1,...",0.0,0.0,NaN,NaN,1.0
0,log1_sim_vcs.log,Simulation,VCS,ALU,0.0,0.0,65.0,Error-[DIV0] Division by zero detected in alu....,0.0,0.0,NaN,NaN,0.0
2,log3_sim_vcs.log,Simulation,VCS,Round Robin Arbiter,0.0,0.0,68.0,"Errors, 2 Warnings",0.0,0.0,NaN,NaN,0.0
